# Data Analysis and Q&A Project Using a Local LLM

## Project Overview

This project requires you to perform a comprehensive analysis of a company's stock data using only the provided data sources and a local LLM. Your analysis should answer the following six questions strictly based on the supplied data and documents—no external data is allowed. All generated answers must be firmly based on the provided data, without any fabricated content. In addition, your logic must be clear, and any attribution of events must be causally linked.

---

## Provided Data

You will be provided with the following data sets:

#### Stock Price Data (Json format)
* Timeframe: Jan 22 to Feb 5
* Fields: Open, High, Low, Close, Volume

#### Quarterly Earnings Data for the Past Year (Json format)
* Contains key financial indicators (e.g., revenue, eps) for each quarter.

#### Full Earnings Transcript Call
* The complete transcript of the earnings call, including management discussions and Q&A.

#### Balance Sheet Data for the Past Year (Json format)
* Includes assets, liabilities, and shareholders' equity information.

#### News Articles
* Full text of 10 news articles related to the company during the analysis period.

---

## Questions
Using the provided data and a local LLM, you need to answer the following six questions:

1. What is the performance of the Tesla stock during this period (Jan 22 to Feb 5)?

2. Why did the price increase on Jan 30? Please provide potential factors.

3. Compared with previous quarters, how is the performance of this quarter?

4. With unsupervised Full Self Driving scheduled to launch in limited markets like Austin by June, what regulatory challenges does Tesla foresee for a nationwide or international rollout, and how is the company strategically preparing to address these hurdles?

5. What insights can be concluded from the earnings call?

6. Which key news events influenced the stock performance, and what insights do they offer?

---

## Project Requirements
- #### Data Source Restriction:
Only use the provided data and documents. No external data or information is allowed.

- #### Answer Generation:
All generated answers must strictly be based on the provided data and documents. The LLM should not "invent" information.

- #### Clear Logic and Causal Relationships:
For each question, your answers must clearly demonstrate logical reasoning, and any attribution of cause must be explicitly linked to events in the data.

- #### Prompt Design:
You must design your own prompts for calling the local LLM to ensure that the responses are generated strictly based on the analysis results.

- #### Result Evaluation:
After generating the answers, implement an evaluation step to assess whether the responses meet the above requirements in terms of data reliance, logical clarity, and correct causation.

- #### Please put the answers to these 6 questions in a dict at the end of your submitted Python nodebook file.

For example
```code
{ "Q1 answer": "Answer1", "Q2 answer": "Answer2", "Q3 answer": "Answer3", "Q4 answer4": "Answer4", "Q5 answer": "Answer5", "Q6 answer": "Answer6"}
```

## Dependencies

* Transformers
* Torch (PyTorch)
* Accelerate

In [27]:
%pip install transformers accelerate pandas ipython
# %pip install torch # Install PyTorch if you dont have it downloading 

DATA_DIR = "447_dataset"

import pandas as pd
import json
import os
from IPython.display import Markdown, display
import datetime


Note: you may need to restart the kernel to use updated packages.


## Loading and Running the Local LLM

1. **Imports Transformers utilities**  
   - `AutoModelForCausalLM`: generic class for loading any GPT‑style model  
   - `AutoTokenizer`: matching tokenizer for converting text ↔ tokens  
   - `pipeline`: high‑level helper that ties model + tokenizer into one callable  

2. **Specifies the model repository**  
   ```python
   model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="cuda"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

# The pipeline will automatically use the model and tokenizer you just loaded
tokenizer = AutoTokenizer.from_pretrained(model_path) # Load the tokenizer

 # Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 2000, # Limit the number of tokens generated
    "return_full_text": False, # Return only the generated text
    "do_sample": True, # Use sampling to generate text
    "temperature": 0.1,# Control the randomness of the output
    "repetition_penalty": 1.1,
    "top_p": 0.9, # Control the diversity of the output
    "top_k": 50, # Control the diversity of the output
}

/Users/chathilaratnatilake/Documents/University/WINTER 2025/ECE 447/Stock-Data-Analyzer/project-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
general_system_prompt = ( 
    "You are an expert financial data analyst LLM.\n"
    "You must ensure that bellow rules are followed:\n"
    "1. Use only the data summaries provided in the prompt\n"
    "2. Show clear step-by-step reasoning , linking each claim to the data.\n"
    "3. The explanation is clear, logical, and accurate.\n"
    "4. Do not invent or hallucinate any information.\n"
    "After your answer, provide a checklist summary indicating whether each criterion is satisfied. "
    "If any criterion is not met, include a brief explanation."
)


def trim_reasoning(raw):
    tag = "</think>"
    idx = raw.find(tag)

    if idx != -1:
        answer = raw[idx + len(tag):].strip()
    else:
        answer = raw.strip()

    return answer

### Question 1:What is the performance of the Tesla stock during this period (Jan 22 to Feb 5)?

#### Prompt construction

* Here we need to monitor the performance of the tesla stock over the specified days. To monitor the performance of the stock the LLM just need to understand the how the pricing of the stock was through the given period hence why the LLM will need to see the infomation in the `prices.json` file.
* To supplement the LLM to construct its answer we will also show the information in `balancesheet.json` so it can pickup on any trends to as why stock prices deviated and change through the mentioned days

In [ ]:
"""   
This Function extracts the price information and converts the informations
into a simple readable format to be included in the prompt to the LLM
"""
def get_prices_summary():
    prices_path = os.path.join(DATA_DIR, "prices.json")

    # Load raw JSON into a DataFrame
    with open(prices_path, "r") as f:
        prices = pd.DataFrame(json.load(f))

    # Parse dates and index
    prices["Date"] = pd.to_datetime(prices["Date"])
    prices = prices.set_index("Date").sort_index()

    # Build a human-readable summary for each day
    daily_summaries = []
    for date, row in prices.iterrows():
        daily_summaries.append(
            f"{date.strftime('%Y-%m-%d')}: "
            f"Open ${row['Open']:.2f}, "
            f"High ${row['Hight']:.2f}, "
            f"Low ${row['Low']:.2f}, "
            f"Close ${row['Close']:.2f}, "
            f"Volume {int(row['Volume']):,}"
        )

    # Join them into one block of text
    daily_summary_text = "\n".join(daily_summaries)
    return daily_summary_text


print(get_prices_summary())

2025-01-22: Open $416.81, High $428.00, Low $414.59, Close $415.11, Volume 60,963,300
2025-01-23: Open $416.06, High $420.73, Low $408.95, Close $412.38, Volume 50,690,600
2025-01-24: Open $414.45, High $418.88, Low $405.78, Close $406.58, Volume 56,427,100
2025-01-27: Open $394.80, High $406.69, Low $389.00, Close $397.15, Volume 58,125,500
2025-01-28: Open $396.91, High $400.59, Low $386.50, Close $398.09, Volume 48,910,700
2025-01-29: Open $395.21, High $398.59, Low $384.48, Close $389.10, Volume 68,033,600
2025-01-30: Open $410.78, High $412.50, Low $384.41, Close $400.28, Volume 98,092,900
2025-01-31: Open $401.53, High $419.99, Low $401.34, Close $404.60, Volume 83,568,200
2025-02-03: Open $386.68, High $389.17, Low $374.36, Close $383.68, Volume 93,732,100
2025-02-04: Open $382.63, High $394.00, Low $381.40, Close $392.21, Volume 57,072,200
2025-02-05: Open $387.51, High $388.39, Low $375.53, Close $378.17, Volume 57,223,300


In [ ]:
user_prompt_template_q1 =( """
    Here is the summarized Tesla data for Jan 22 - Feb 5:
    {price_info}

    **Question:**  
    What was the performance of Tesla stock over this period?
                          
    **Please:**
    - Write a brief **Introduction** stating the question and data scope.  
    - In your **Analysis**, cite the exact figures (dates, prices, percent changes, volumes).  
    - Draw any causal insights clearly (e.g., “the drop on Feb 1 may be linked to…”).  
    - Finish with a concise **Conclusion** summarizing overall performance.
""")

# Fill in the template with specific data
filled_user_prompt = user_prompt_template_q1.format(
    price_info = get_prices_summary()
)

messages = [
    {"role": "system", "content": general_system_prompt},
    {"role": "user", "content": filled_user_prompt},
]

question1Answer = pipe(messages, **generation_args)
display(Markdown(trim_reasoning(question1Answer[0]['generated_text'])))


NameError: name 'general_system_prompt' is not defined

# Question 2 
### Why did the price increase on Jan 30? Please provide potential factors.

In [9]:
import json

system_prompt_q2 = (
    "You are an expert financial analyst assistant in the United States. "
    "Your job is to explain stock price moves using only the provided data. "
    "You must be concise and give only the most critical information in your answer"
    "Do not invent or fetch any outside information, everything must be grounded in the inputs."
)

user_prompt_q2 = """

Using only the news articles data and Tesla’s Jan 30 stock data, identify all plausible factors that contributed to the price uptick.
Reference the specific news item (by title and date) or data point you’re using.  
Explain the causal link to the price move. 
Do not use any external sources.  

**Question** 
Why did Tesla’s stock price increase on January 30?

Here are the news articles and price info:
{input_data}

"Please respond in English!"
"""

with open("447_dataset/news.json", "r") as f:
    news_data = json.load(f)

news_json_str = json.dumps(news_data)

# Example of how you'd fill & call it:
filled_user = user_prompt_q2.format(input_data=news_json_str)
messages = [
    {"role": "system", "content": system_prompt_q2},
    {"role": "user",   "content": filled_user},
]
output = pipe(messages, **generation_args)
print(output[0]["generated_text"])

</think>

### What Happened?

#### **Shares of Electric Vehicle pioneer Tesla (TSLA) fell 7.1% in the morning session after the Trump administration's new tariffs (25% on Canadian goods and 10% on Chinese products) shook markets.**

#### **Separately, Tesla shares fell 5.1% from $1,000 to $7,394 five years ago.**

---

### **Key Points from the Article**

1. **Tesla shares have risen 1.4% since the beginning of the year, but at $384.48 per share, they are still trading 19.9% below its 52-week high of $479.86 from December 2024.**
   
   - **Wall Street analysts believe Tesla shares will bump higher in mid-Thursday trading after its disappointing fourth-quarter earnings update.**

2. **CEO Elon Musk predicted Tesla's stock would reach $7,394 in mid-Thursday trading.**
   
   - **Musk also emphasized Tesla's long-term vision, saying 'this is the most important year for Tesla,' and 'when we look back on 2025 and the launch of unsupervised Full Self-Driving, true real-world AI that actuall

### Question 3:Compared with previous quarters, how is the performance of this quarter?

#### Data Used for Prompt construction

* For the LLM to have any insight of quaterly financial information, I will provide it the `balancesheet.json` as this file contains the quaterly financial figures. We will use this data so the LLM is a aware of the performance of the previous 1-2 years. This gives the LLM a base to compare against the current financial quater( 2025 Q1). Since the balance sheet has a lot of infomation, we dont want to overload the LLM with too many features to consider when it comes up with its decison so we will handpick some important features from the balance sheet from each quater.


* The Data found in the `earnings.json` also contains relavant financial information for the past five quaters, which the LLM can use compare the performance of the company 

In [36]:
def get_earnings_summary():
    """
    Reads earning.json and returns a plain summary with:
    - EPS predicted / actual
    - Revenue predicted / actual (in billions)
    """
    earnings_path = os.path.join(DATA_DIR, "earning.json")

    with open(earnings_path, "r") as f:
        earnings_json = json.load(f)

    lines = []
    for rec in earnings_json:
        dt = rec.get("EarningReleaseDate") or rec.get("EarningReportDate")
        if not dt:
            continue

        date = pd.to_datetime(dt).strftime("%Y-%m-%d")

        eps_actual   = rec.get("EpsActual", 0)
        eps_forecast = rec.get("EpsForecast", 0)
        eps_surprise   = rec.get("EpsSurprise", 0)

        rev_actual   = rec.get("RevenueActual", 0) / 1e9
        rev_forecast = rec.get("RevenueForecast", 0) / 1e9

        line = (
            f"{date}: "
            f"EPS predicted: {eps_forecast:.2f}, EPS actual: {eps_actual:.2f}, EPS surprise: {eps_surprise:.2f}; "
            f"Revenue predicted: ${rev_forecast:.2f}B, Revenue actual: ${rev_actual:.2f}B"
        )
        lines.append(line)

    return "\n".join(lines)

def get_balance_sheet_summary():
    """
    This function extracts financial statement information from balencesheet.json
    and returns a readable summary line for each quarter.
    """
    bs_path = os.path.join(DATA_DIR, "balencesheet.json")

    # Load raw JSON into a DataFrame
    with open(bs_path, "r") as f:
        balance_sheet = pd.DataFrame(json.load(f))

    # Parse dates and sort
    balance_sheet["Date"] = pd.to_datetime(balance_sheet["Date"])
    balance_sheet = balance_sheet.set_index("Date").sort_index()

    # Build human-readable lines
    summaries = []
    for date, row in balance_sheet.iterrows():
        rev       = row.get("Total Revenue", 0) / 1e6
        gp        = row.get("Gross Profit", 0) / 1e6
        op_inc    = row.get("Operating Income", 0) / 1e6
        net_inc   = row.get("Net Income Common Stockholders", 0) / 1e6
        ebitda    = row.get("EBITDA", 0) / 1e6
        tot_exp   = row.get("Total Expenses", 0) / 1e6
        liab      = row.get("Total Liabilities", 0) / 1e6
        equity    = row.get("Total Shareholders’ Equity", row.get("Total Shareholder Equity", 0)) / 1e6

        # Calculate margins and ratios safely
        gross_margin   = gp / rev * 100 if rev else 0
        op_margin      = op_inc / rev * 100 if rev else 0
        net_margin     = net_inc / rev * 100 if rev else 0

        summaries.append(
            f"{date.strftime('%Y-%m-%d')}: "
            f"Revenue ${rev:.1f}M, Gross Profit ${gp:.1f}M ({gross_margin:.1f}%), "
            f"Operational Income ${op_inc:.1f}M ({op_margin:.1f}%), Net Income ${net_inc:.1f}M ({net_margin:.1f}%), "
            f"EBITDA ${ebitda:.1f}M, Total Expenses ${tot_exp:.1f}M, "
        )

    return "\n".join(summaries)

print("----------Balance Sheet Summary----------")
print(get_balance_sheet_summary())
print("\n-----------Earnings Summary---------------")
print(get_earnings_summary())

----------Balance Sheet Summary----------
2023-12-31: Revenue $25.2M, Gross Profit $4.4M (17.6%), Operational Income $2.1M (8.2%), Net Income $7.9M (31.5%), EBITDA $3.5M, Total Expenses $23.1M, 
2024-03-31: Revenue $21.3M, Gross Profit $3.7M (17.4%), Operational Income $1.2M (5.5%), Net Income $1.2M (5.5%), EBITDA $2.9M, Total Expenses $20.1M, 
2024-06-30: Revenue $25.5M, Gross Profit $4.6M (18.0%), Operational Income $2.2M (8.7%), Net Income $1.5M (5.8%), EBITDA $3.3M, Total Expenses $23.3M, 
2024-09-30: Revenue $25.2M, Gross Profit $5.0M (19.8%), Operational Income $2.8M (11.0%), Net Income $2.2M (8.6%), EBITDA $4.2M, Total Expenses $22.4M, 
2024-12-31: Revenue $25.7M, Gross Profit $4.2M (16.3%), Operational Income $1.6M (6.2%), Net Income $2.3M (9.0%), EBITDA $4.4M, Total Expenses $24.1M, 

-----------Earnings Summary---------------
2024-01-24: EPS predicted: 0.74, EPS actual: 0.71, EPS surprise: -0.03; Revenue predicted: $25.76B, Revenue actual: $25.17B
2024-04-23: EPS predicted: 0

In [ ]:
user_prompt = f"""
Here are your two data summaries:

**Balance Sheet Summary** 
- Balance sheet summary containing the quaterly financial information for the previous quaters 
{get_balance_sheet_summary()}


**Earnings Summary**  
- Earnings summary containing the quaterly earnings information for the previous quaters and the current quater(2025 Q1)
{get_earnings_summary()}

**Question:**  
Compared with previous quarters, how is the performance of this quarter (ending 2025‑01‑29)?

**Please:**  
- Write a brief **Introduction** stating the question and data scope.  
- In your **Analysis**, compare each metric (Revenue, Gross Profit, Operational Income, Net Income, EBITDA, EPS and Revenue surprises) quarter‑over‑quarter, citing the exact figures.  
- Highlight any notable trends (e.g., margin expansions or EPS misses).  
- Finish with a concise **Conclusion** summarizing whether performance improved or deteriorated and why.
"""

messages = [
    {"role": "system", "content": general_system_prompt},
    {"role": "user",   "content": user_prompt},
]

question3Answer = pipe(messages, **generation_args)
display(Markdown(trim_reasoning(question3Answer[0]['generated_text'])))